In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "twitter_toxic_tweets.csv"

df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "umitka/twitter-toxic-tweets",
    file_path
)

print(df.head())


In [ ]:
from sklearn.model_selection import train_test_split


X = df['tweet']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
print(X_train.shape)
print(y_train.shape)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words='english'
)

X_train = vectorizer.fit_transform(X_train).toarray()
X_test = vectorizer.transform(X_test).toarray()


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)


Epoch 1/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 21s 30ms/step - accuracy: 0.9305 - loss: 0.2669 - val_accuracy: 0.9531 - val_loss: 0.1384
Epoch 2/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 15s 24ms/step - accuracy: 0.9647 - loss: 0.0926 - val_accuracy: 0.9546 - val_loss: 0.1424
Epoch 3/10
156/640 ━━━━━━━━━━━━━━━━━━━━ 13s 27ms/step - accuracy: 0.9800 - loss: 0.0557

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()
